# Refund Decision Simulator
### Rule-Based vs Machine Learning Decision Systems

This experiment simulates refund decisions using synthetic order data
and compares a heuristic rule-based system with a Logistic Regression model.

The evaluation includes:
- Accuracy
- Confusion Matrix
- Classification Report
- Economic cost simulation


n = 1000

order_amount = np.random.uniform(100, 2000, n)
delay_minutes = np.random.randint(0, 90, n)
previous_refunds = np.random.randint(0, 5, n)
fraud_score = np.random.uniform(0, 1, n)
complaint_severity = np.random.randint(1, 6, n)

refund_prob = (
    0.3 * (delay_minutes > 30) +
    0.4 * (complaint_severity > 3) -
    0.5 * (fraud_score > 0.7)
)

refund_prob = 1 / (1 + np.exp(-refund_prob))
refunded = np.random.binomial(1, refund_prob)

data = pd.DataFrame({
    "order_amount": order_amount,
    "delay_minutes": delay_minutes,
    "previous_refunds": previous_refunds,
    "fraud_score": fraud_score,
    "complaint_severity": complaint_severity,
    "refunded": refunded
})

data.head()


def rule_based(row):
    if row["delay_minutes"] > 40 and row["fraud_score"] < 0.7:
        return 1
    if row["complaint_severity"] > 4:
        return 1
    return 0

data["rule_prediction"] = data.apply(rule_based, axis=1)

rule_accuracy = accuracy_score(data["refunded"], data["rule_prediction"])

print("Rule-Based Accuracy:", rule_accuracy)
print("Rule-Based Confusion Matrix:")
print(confusion_matrix(data["refunded"], data["rule_prediction"]))


In [ ]:
X = data.drop(columns=["refunded", "rule_prediction"])
y = data["refunded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

ml_predictions = model.predict(X_test)

ml_accuracy = accuracy_score(y_test, ml_predictions)

print("ML Accuracy:", ml_accuracy)
print("ML Confusion Matrix:")
print(confusion_matrix(y_test, ml_predictions))
print("\nML Classification Report:")
print(classification_report(y_test, ml_predictions))


ML Accuracy: 0.565
ML Confusion Matrix:
[[38 59]
 [28 75]]

ML Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.39      0.47        97
           1       0.56      0.73      0.63       103

    accuracy                           0.56       200
   macro avg       0.57      0.56      0.55       200
weighted avg       0.57      0.56      0.55       200



In [ ]:
retention_value = 500
fraud_threshold = 0.7

def calculate_cost(df, predictions):
    total_cost = 0
    for i, pred in enumerate(predictions):
        row = df.iloc[i]
        if pred == 1:
            total_cost += row["order_amount"]
            if row["fraud_score"] > fraud_threshold:
                total_cost += row["order_amount"]
        else:
            if row["refunded"] == 1:
                total_cost += retention_value
    return total_cost

rule_cost = calculate_cost(data, data["rule_prediction"])
ml_cost = calculate_cost(data.loc[X_test.index], ml_predictions)

print("Total Economic Cost (Rule-Based):", rule_cost)
print("Total Economic Cost (ML-Based):", ml_cost)


Total Economic Cost (Rule-Based): 707866.6807319408
Total Economic Cost (ML-Based): 158277.1661673083
